# RAG with Qwen, FAISS, PDFs, and Spreadsheets

This notebook is designed for teaching  how **Retrieval-Augmented Generation (RAG)** works in practice.

They will see:

1. How we turn text into embeddings (vectors that represent meaning).
2. How we store those vectors in a **vector database** (FAISS).
3. How we **retrieve** relevant information for a question.
4. How we give that information to a language model (Qwen) to get grounded answers.
5. How to upload **PDFs** and **spreadsheets** and make them part of the knowledge base.


## What is RAG?

**RAG = Retrieval-Augmented Generation.**

The idea:

1. A user asks a question.
2. We **retrieve** relevant pieces of text from our documents.
3. We feed those pieces + the question into a language model.
4. The model **generates** an answer that uses the retrieved context.

Without RAG, the model is guessing from what it learned during pretraining.
With RAG, the model is allowed to "look things up" in our own data first.

Diagram:

```text
User question
      |
      v
+-------------------+
|  Embedding model  |  -> question vector
+-------------------+
      |
      v
+------------------------+
|  Vector database       |  -> find similar document vectors
|      (FAISS)           |
+------------------------+
      |
      v
+----------------------+
|  Retrieved context   |
+----------------------+
      |
      v
+------------------------------+
|  Language model (Qwen)       |
+------------------------------+
      |
      v
Answer, using your documents
```


## Step 1 — Install Dependencies

We install:

- `transformers` + `accelerate` to load and run Qwen.
- `sentence-transformers` for the embedding model.
- `faiss-cpu` for vector search.
- `pypdf` to extract text from PDFs.
- `pandas` + `openpyxl` to read spreadsheets (CSV/XLSX).


In [ ]:
!pip install -q transformers accelerate sentence-transformers faiss-cpu pypdf pandas openpyxl

## Step 2 — Load the Embedding Model

We use `BAAI/bge-small-en-v1.5` as our embedding model.

What this model does:

- Input: text (a sentence, paragraph, or document).
- Output: a **vector** (a list of numbers) that represents the meaning of that text.

Similar meanings → similar vectors.

Diagram:

```text
Text: "Python is a popular programming language."

         Embedding model
   +-------------------------+
   | BGE-small-en-v1.5       |
   +-------------------------+
                 |
                 v
Vector: [ -0.12, 0.88, 0.31, -0.44, ..., 0.05 ]

Another sentence like:
"Python is widely used in AI and data science."

produces a vector that is close in this high-dimensional space.
```


In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("BAAI/bge-small-en-v1.5")
print("Embedding model loaded!")

# Detect the embedding dimension dynamically so FAISS always matches.
EMBED_DIM = embedder.get_sentence_embedding_dimension()
print("Embedding dimension:", EMBED_DIM)

## Step 3 — Load the Qwen Language Model

We load a small Qwen chat model.

Key points for students:

- Qwen is the **generator**: it turns text prompts into answers.
- It does **not** store your PDFs or spreadsheets internally.
- It works best when we give it the right context (via RAG).

We use the `dtype` parameter (recommended) for the tensor type.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "Qwen/Qwen1.5-1.8B-Chat"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.float32,
    device_map="auto"
)

print("Qwen model loaded!")

## Step 4 — Create the FAISS Vector Index

Now we set up a **vector database** using FAISS.

What FAISS does:

- Stores many embedding vectors.
- Given a new vector (from a question), quickly finds the most similar stored vectors.
- Those matches correspond to our most relevant documents.

Diagram:

```text
         FAISS index (vector database)
       +------------------------------+
       |  vec_0  -> doc_0 (text)      |
       |  vec_1  -> doc_1 (text)      |
       |  vec_2  -> doc_2 (text)      |
       |   ...                        |
       +------------------------------+
```


In [ ]:
import faiss
import numpy as np

# Create a simple L2 index with the correct dimension
index = faiss.IndexFlatL2(EMBED_DIM)

# Store raw texts and simple metadata
documents = []   # list of text
sources = []     # where each text came from (manual, pdf filename, spreadsheet name, etc.)

print("FAISS index created with dimension:", EMBED_DIM)

## Step 5 — Helper Functions (Add Documents, PDFs, Spreadsheets)

To avoid missing definitions, **all core helper functions are defined in one cell**:

- `add_document(text, source)`
- `upload_pdf()`
- `upload_spreadsheet()`

This way, students only need to run this one cell to get all core behaviors.


In [ ]:
from typing import List
from pypdf import PdfReader
import pandas as pd
from google.colab import files


def add_document(text: str, source: str = "manual"):
    """Embed the text, add it to FAISS, and record its source.

    text   : the raw text to store
    source : a short label like 'manual', 'myfile.pdf', or 'grades.xlsx'
    """
    if not text.strip():
        print("Skipped empty text.")
        return

    embedding = embedder.encode([text])[0].astype("float32")
    index.add(np.array([embedding]))
    documents.append(text)
    sources.append(source)
    print(f"Added document #{len(documents)} from source: {source} (length={len(text)} chars)")


def upload_pdf():
    """Upload one or more PDFs, extract all text, and add each as a document.

    Each uploaded PDF becomes one document in FAISS.
    For more advanced use, you could split large PDFs into chunks.
    """
    uploaded = files.upload()
    for filename in uploaded.keys():
        reader = PdfReader(filename)
        full_text = ""
        for page in reader.pages:
            page_text = page.extract_text() or ""
            full_text += page_text + "\n"
        print(f"[PDF] Extracted {len(full_text)} characters from {filename}")
        add_document(full_text, source=filename)


def upload_spreadsheet():
    """Upload one or more spreadsheets (CSV or Excel) and add them as text documents.

    Strategy:
    - If CSV: read with pandas.read_csv.
    - If Excel: read with pandas.read_excel (requires openpyxl).
    - Convert DataFrame to CSV-style text string.
    - Store that text in FAISS so it can be retrieved semantically.
    """
    uploaded = files.upload()
    for filename in uploaded.keys():
        if filename.lower().endswith(".csv"):
            df = pd.read_csv(filename)
        else:
            df = pd.read_excel(filename)
        text = df.to_csv(index=False)
        print(f"[Spreadsheet] {filename}: {df.shape[0]} rows, {df.shape[1]} columns")
        add_document(text, source=filename)


print("Helper functions defined: add_document(), upload_pdf(), upload_spreadsheet()")

## Step 6 — Add Some Sample Documents

We add a few short texts so students can immediately test the system.

Later, they can:

- Upload their own PDFs.
- Upload their own spreadsheets.
- See how retrieval and answers change.


In [ ]:
add_document("Python is a high-level programming language widely used in data science and AI.", source="sample: python")
add_document("Retrieval-Augmented Generation (RAG) retrieves relevant documents before the model generates an answer.", source="sample: rag")
add_document("Google Cloud provides scalable infrastructure for machine learning, storage, and databases.", source="sample: gcp")

## Step 7 — RAG Query Function

This function runs the full RAG pipeline:

1. Embed the question.
2. Use FAISS to find the most similar documents.
3. Build a **context** string from those documents.
4. Construct a prompt that includes the context and the question.
5. Ask Qwen to generate an answer.

Diagram:

```text
Question
  |
  v
Embedding model --> question vector
  |
  v
FAISS index --> indices of closest documents
  |
  v
Context = joined documents
  |
  v
Qwen model --> final answer
```


In [ ]:
def rag_query(query: str, top_k: int = 3, show_sources: bool = True) -> str:
    # 1. Embed the query
    q_embedding = embedder.encode([query])[0].astype("float32")

    # 2. Search FAISS
    distances, indices = index.search(np.array([q_embedding]), top_k)

    # 3. Build context from the closest documents
    retrieved_docs = []
    retrieved_meta = []
    for idx in indices[0]:
        if 0 <= idx < len(documents):
            retrieved_docs.append(documents[idx])
            retrieved_meta.append(sources[idx])

    context = "\n\n---\n\n".join(retrieved_docs)

    if show_sources:
        print("Retrieved from sources:")
        for m in retrieved_meta:
            print(" -", m)
        print()

    # 4. Build prompt
    prompt = f"""You are a helpful assistant.

You will be given some context from documents, followed by a question.
Use only the information in the context to answer as accurately as possible.

Context:
{context}

Question: {query}

Answer:
"""

    # 5. Generate answer with Qwen
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=0.2
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

## Step 8 — Test the System

Now we can ask a question and see:

- Which sources FAISS retrieved.
- How Qwen answers using that context.


In [ ]:
print(rag_query("What is Q-Mos?"))

In [ ]:
upload_pdf()


## Step 9 —  Exercises

Things to try:

1. **Upload a PDF** using `upload_pdf()` and then ask questions about it.
2. **Upload a spreadsheet** using `upload_spreadsheet()` and ask questions like:
   - "Which region has the highest sales?"
   - "Which students have the lowest grades?"
3. Modify `top_k` in `rag_query` and observe how the answer changes.
4. Print out the raw `context` inside `rag_query` to see exactly what the model saw.
5. Split large documents into smaller logical chunks (per page, per section) and store each chunk separately for more precise retrieval.

This helps you understand:

- The difference between **raw data** and **retrieved context**.
- How better retrieval leads to better answers.
- Why vector databases and embeddings are powerful in modern data science and AI systems.
